# Setting up train and test data for Model evaluation

In [ ]:
Df_train = pd.read_csv(r'Train_set_knn_1.csv')
Df_test = pd.read_csv(r'Test_set_knn_1.csv')

cols = [c for c in Df_train.columns if c!= 'Actual op']
x_train = Df_train[cols]
x_test = Df_test[cols]
y_train = Df_train['Actual op']
y_test = Df_test['Actual op']

del x_train['Unnamed: 0']
del x_test['Unnamed: 0']

# Settingup the pipelines for different Classification models

## With hyperparameter tuning

In [ ]:
cv = KFold(n_splits=5, shuffle=True)

parameters_knn = {'n_neighbors':[3,5,11,19], 'weights':['uniform','distance'], 'metric':['euclidean','manhattan']}
parameters_random_forest = {'n_estimators': [500,800],'max_depth' : [8,12,16,20]}
parameters_descision_tree = {'min_samples_split' : range(10,30,1),'max_depth': range(10,25,2)}
parameters_svm_linear = {'gamma':np.logspace(-6,-2,5), 'C':np.logspace(0,2,10)}
parameters_svm_rbf = {'C':np.logspace(1,2,5)}
parameters_logestic = {'penalty' = ['l1','l2'],'C':np.logspace(0,2,5)}

KNN = make_pipeline(
    StandardScaler(),
    GridSearchCV(estimator=KNeighborsClassifier(),param_grid = parameters_knn,scoring = 'f1',cv=cv)
    
)

Random_Forest = make_pipeline(
StandardScaler(),
GridSearchCV(estimator=RandomForestClassifier(), param_grid=parameters_ramdom_forest, cv= cv)
)


Descision_tree = make_pipeline(
    StandardScaler(),
    GridSearchCV(estimator=DecisionTreeClassifier(),param_grid = parameters_descision_tree,scoring='f1',cv= cv)
)

SVM_linear = make_pipeline(
    StandardScaler(),
    GridSearchCV(estimator=SVC(kernel='linear',max_iter = 10000),param_grid = parameters_svm_linear,scoring = 'f1',cv=cv)
)

SVM_rbf = make_pipeline(
    StandardScaler(),
    GridSearchCV(estimator=SVC(kernel='rbf',max_iter = 10000),param_grid = parameters_rbf,scoring = 'f1',cv=cv)
)

Logestic_regression = make_pipeline(
    StandardScaler(),
    GridSearchCV(estimator=LogisticRegression(),param_grid = parameters_logestic,scoring = 'f1',cv=cv)
)

# Making a function that returns best parameters and the cross validated scores

In [ ]:
def score_model(model):
    score = cross_val_score(KNN_pipe.fit(x_train,y_train), x_train, y_train,scoring='f1', cv=cv)
    best_par = KNN_pipe[1].best_params_
    return np.mean(score),best_par

In [ ]:
models = [KNN,Random_Forest,Descision_tree,SVM_linear,SVM_rbf,Logestic_regression]

In [ ]:
scores_models = []
parameters_models = []
for model in models:
    Score,Parameters = score_model(model)
    scores_models.append(Score)
    parameters_models.append(Parameters)